# Automated doubly robust estimation with ChiRho - TMLE Version

## Outline

- [Setup](#setup)

- [Overview: Systematically adjusting for observed confounding](#overview:-systematically-adjusting-for-observed-confounding)
    - [Task: Treatment effect estimation with observational data](#task:-treatment-effect-estimation-with-observational-data)
    - [Challenge: Confounding](#challenge:-confounding)
    - [Assumptions: All confounders observed](#assumptions:-all-confounders-observed)
    - [Intuition: Statistically adjusting for confounding](#intuition:-statistically-adjusting-for-confounding)

- [Causal Probabilistic Program](#causal-probabilistic-program)
    - [Model description](#model-description)
    - [Generating data](#generating-data)
    - [Fit parameters via maximum likelihood](#fit-parameters-via-maximum-likelihood)

- [Causal Query: average treatment effect (ATE)](#causal-query:-average-treatment-effect-\(ATE\))
    - [Defining the target functional](#defining-the-target-functional)
    - [Closed form doubly robust correction](#closed-form-doubly-robust-correction)
    - [Computing automated doubly robust correction via Monte Carlo](#computing-automated-doubly-robust-correction-via-monte-carlo)
    - [Results](#results)

- [References](#references)

## Setup

Here, we install the necessary Pytorch, Pyro, and ChiRho dependencies for this example.

In [ ]:
from typing import Callable, Optional, Tuple

import functools
import torch
import math
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pyro
import pyro.distributions as dist
from pyro.infer import Predictive
import pyro.contrib.gp as gp

from chirho.counterfactual.handlers import MultiWorldCounterfactual
from chirho.indexed.ops import IndexSet, gather
from chirho.interventional.handlers import do
from chirho.robust.internals.utils import ParamDict
from chirho.robust.handlers.estimators import one_step_corrected_estimator, tmle
from chirho.robust.handlers.predictive import PredictiveModel 
from chirho.robust.ops import influence_fn

pyro.settings.set(module_local_params=True)

sns.set_style("white")

pyro.set_rng_seed(321) # for reproducibility

## Overview

In this tutorial, we will use ChiRho to estimate the average treatment effect (ATE) from observational data. We will use a simple example to illustrate the basic concepts of doubly robust estimation and how ChiRho can be used to automate the process for more general summaries of interest. 

There are five main steps to our doubly robust estimation procedure but only the last step is different from a standard probabilistic programming workflow:
1. Write model of interest
    - Define probabilistic model of interest using Pyro
2. Feed in data
    - Observed data used to train the model
3. Run inference
    - Use Pyro's rich inference library to fit the model to the data
4. Define target functional
    - This is the model summary of interest (e.g. average treatment effect)
5. Compute robust estimate
    - Use ChiRho to compute the doubly robust estimate of the target functional
    - Importantly, this step is automated and does not require refitting the model for each new functional


Our proposed automated robust inference pipeline is summarized in the figure below.

![fig1](figures/robust_pipeline.png)

## Causal Probabilistic Program

### Model Description
In this example, we will focus on a cannonical model `CausalGLM` consisting of three types of variables: binary treatment (`A`), confounders (`X`), and response (`Y`). For simplicitly, we assume that the response is generated from a generalized linear model with link function $g$. The model is described by the following generative process:

$$
\begin{align*}
X &\sim \text{Normal}(0, I_p) \\
A &\sim \text{Bernoulli}(\pi(X)) \\
\mu &= \beta_0 + \beta_1^T X + \tau A \\
Y &\sim \text{ExponentialFamily}(\text{mean} = g^{-1}(\mu))
\end{align*}
$$

where $p$ denotes the number of confounders, $\pi(X)$ is the probability of treatment conditional on confounders $X$, $\beta_0$ is the intercept, $\beta_1$ is the confounder effect, and $\tau$ is the treatment effect.

In [ ]:
class CausalGLM(pyro.nn.PyroModule):
    def __init__(
        self,
        p: int,
        link_fn: Callable[..., dist.Distribution] = lambda mu: dist.Normal(mu, 1.0),
        prior_scale: Optional[float] = None,
    ):
        super().__init__()
        self.p = p
        self.link_fn = link_fn
        if prior_scale is None:
            self.prior_scale = 1 / math.sqrt(self.p)
        else:
            self.prior_scale = prior_scale

    def sample_outcome_weights(self):
        return pyro.sample(
            "outcome_weights",
            dist.Normal(0.0, self.prior_scale).expand((self.p,)).to_event(1),
        )

    def sample_intercept(self):
        return pyro.sample("intercept", dist.Normal(0.0, 1.0))

    def sample_propensity_weights(self):
        return pyro.sample(
            "propensity_weights",
            dist.Normal(0.0, self.prior_scale).expand((self.p,)).to_event(1),
        )

    def sample_treatment_weight(self):
        return pyro.sample("treatment_weight", dist.Normal(0.0, 1.0))

    def sample_covariate_loc_scale(self):
        return torch.zeros(self.p), torch.ones(self.p)

    def forward(self):
        intercept = self.sample_intercept()
        outcome_weights = self.sample_outcome_weights()
        propensity_weights = self.sample_propensity_weights()
        tau = self.sample_treatment_weight()
        x_loc, x_scale = self.sample_covariate_loc_scale()
        X = pyro.sample("X", dist.Normal(x_loc, x_scale).to_event(1))
        A = pyro.sample(
            "A",
            dist.Bernoulli(
                logits=torch.einsum("...i,...i->...", X, propensity_weights)
            ),
        )

        return pyro.sample(
            "Y",
            self.link_fn(
                torch.einsum("...i,...i->...", X, outcome_weights) + A * tau + intercept
            ),
        )

Next, we will condition on both treatment and confounders to estimate the causal effect of treatment on the outcome. We will use the following causal probabilistic program to do so:

In [ ]:
class ConditionedCausalGLM(CausalGLM):
    def __init__(
        self,
        X: torch.Tensor,
        A: torch.Tensor,
        Y: torch.Tensor,
        link_fn: Callable[..., dist.Distribution] = lambda mu: dist.Normal(mu, 1.0),
        prior_scale: Optional[float] = None,
    ):
        p = X.shape[1]
        super().__init__(p, link_fn, prior_scale)
        self.X = X
        self.A = A
        self.Y = Y

    def forward(self):
        intercept = self.sample_intercept()
        outcome_weights = self.sample_outcome_weights()
        propensity_weights = self.sample_propensity_weights()
        tau = self.sample_treatment_weight()
        x_loc, x_scale = self.sample_covariate_loc_scale()
        with pyro.plate("__train__", size=self.X.shape[0], dim=-1):
            X = pyro.sample("X", dist.Normal(x_loc, x_scale).to_event(1), obs=self.X)
            A = pyro.sample(
                "A",
                dist.Bernoulli(
                    logits=torch.einsum("ni,i->n", self.X, propensity_weights)
                ),
                obs=self.A,
            )
            pyro.sample(
                "Y",
                self.link_fn(
                    torch.einsum("ni,i->n", X, outcome_weights)
                    + A * tau
                    + intercept
                ),
                obs=self.Y,
            )

In [ ]:
# Visualize the model
pyro.render_model(
    ConditionedCausalGLM(torch.zeros(1, 1), torch.zeros(1), torch.zeros(1)),
    render_params=True, 
    render_distributions=True
)

### Generating data

For evaluation, we generate `N_datasets` datasets, each with `N` samples. We compare vanilla estimates of the target functional with the double robust estimates of the target functional across the `N_sims` datasets. We use a similar data generating process as in Kennedy (2022).

In [ ]:
class GroundTruthModel(CausalGLM):
    def __init__(
        self,
        p: int,
        alpha: int,
        beta: int,
        link_fn: Callable[..., dist.Distribution] = lambda mu: dist.Normal(mu, 1.0),
        treatment_weight: float = 0.0,
    ):
        super().__init__(p, link_fn)
        self.alpha = alpha  # sparsity of propensity weights
        self.beta = beta  # sparsity of outcome weights
        self.treatment_weight = treatment_weight

    def sample_outcome_weights(self):
        outcome_weights = 1 / math.sqrt(self.beta) * torch.ones(self.p)
        outcome_weights[self.beta :] = 0.0
        return outcome_weights

    def sample_propensity_weights(self):
        propensity_weights = 1 / math.sqrt(self.alpha) * torch.ones(self.p)
        propensity_weights[self.alpha :] = 0.0
        return propensity_weights

    def sample_treatment_weight(self):
        return torch.tensor(self.treatment_weight)

    def sample_intercept(self):
        return torch.tensor(0.0)

In [ ]:
N_datasets = 100
simulated_datasets = []

# Data configuration
p = 200
alpha = 50
beta = 50
N_train = 500
N_test = 500

true_model = GroundTruthModel(p, alpha, beta)
prior_model = CausalGLM(p)

for _ in range(N_datasets):
    # Generate data
    D_train = Predictive(
        true_model, num_samples=N_train, return_sites=["X", "A", "Y"]
    )()
    D_test = Predictive(
        true_model, num_samples=N_test, return_sites=["X", "A", "Y"]
    )()
    simulated_datasets.append((D_train, D_test))

### Fit parameters via maximum likelihood

In [ ]:
fitted_params = []
for i in range(N_datasets):
    # Generate data
    D_train = simulated_datasets[i][0]

    # Fit model using maximum likelihood
    conditioned_model = ConditionedCausalGLM(
        X=D_train["X"], A=D_train["A"], Y=D_train["Y"]
    )
    
    guide_train = pyro.infer.autoguide.AutoDelta(conditioned_model)
    elbo = pyro.infer.Trace_ELBO()(conditioned_model, guide_train)

    # initialize parameters
    elbo()
    adam = torch.optim.Adam(elbo.parameters(), lr=0.03)

    # Do gradient steps
    for _ in range(2000):
        adam.zero_grad()
        loss = elbo()
        loss.backward()
        adam.step()

    theta_hat = {
        k: v.clone().detach().requires_grad_(True) for k, v in guide_train().items()
    }
    fitted_params.append(theta_hat)

## Causal Query: Average treatment effect (ATE)

The average treatment effect summarizes, on average, how much the treatment changes the response, $ATE = \mathbb{E}[Y|do(A=1)] - \mathbb{E}[Y|do(A=0)]$. The `do` notation indicates that the expectations are taken according to *intervened* versions of the model, with $A$ set to a particular value. Note from our [tutorial](tutorial_i.ipynb) that this is different from conditioning on $A$ in the original `causal_model`, which assumes $X$ and $T$ are dependent.


To implement this query in ChiRho, we define the `ATEFunctional` class which take in a `model` and `guide` and returns the average treatment effect by simulating from the posterior predictive distribution of the model and guide.

### Defining the target functional

In [ ]:
class ATEFunctional(torch.nn.Module):
    def __init__(self, model: Callable, *, num_monte_carlo: int = 100):
        super().__init__()
        self.model = model
        self.num_monte_carlo = num_monte_carlo
    
    def forward(self, *args, **kwargs):
        with MultiWorldCounterfactual():
            with pyro.plate("monte_carlo_functional", size=self.num_monte_carlo, dim=-2):
                with do(actions=dict(A=(torch.tensor(0.0), torch.tensor(1.0)))):
                    Ys = self.model(*args, **kwargs)
                Y0 = gather(Ys, IndexSet(A={1}), event_dim=0)
                Y1 = gather(Ys, IndexSet(A={2}), event_dim=0)
        ate = (Y1 - Y0).mean(dim=-2, keepdim=True).mean(dim=-1, keepdim=True).squeeze()
        return pyro.deterministic("ATE", ate)

### Closed form doubly robust correction

For the average treatment effect functional, there exists a closed-form analytical formula for the doubly robust correction. This formula is derived in Kennedy (2022) and is implemented below:

In [ ]:
from typing import Any
from chirho.robust.ops import Functional, Point, P, S, T

def ATECausalGLM_analytic_influence(functional: Functional[P, S], 
                                    point: Point[T], 
                                    pointwise_influence: bool = True,
                                    **kwargs) -> Functional[P, S]:
    # assert isinstance(functional, ATEFunctional)
    def new_functional(model: Callable[P, Any]) -> Callable[P, S]:
        assert isinstance(model.model, CausalGLM)
        theta = dict(model.guide.named_parameters())
        def new_model(*args, **kwargs):
            X = point["X"]
            A = point["A"]
            Y = point["Y"]
            
            pi_X = torch.sigmoid(torch.einsum("...i,...i->...", X, theta["propensity_weights_param"]))
            mu_X = (
                torch.einsum("...i,...i->...", X, theta["outcome_weights_param"])
                + A * theta["treatment_weight_param"]
                + theta["intercept_param"]
            )
            analytic_eif_at_pts = (A / pi_X - (1 - A) / (1 - pi_X)) * (Y - mu_X)
            if pointwise_influence:
                return analytic_eif_at_pts
            else:
                return analytic_eif_at_pts.mean()


        return new_model
    return new_functional


In [ ]:
# # Closed form expression
# def closed_form_doubly_robust_ate_correction(X_test, theta) -> Tuple[torch.Tensor, torch.Tensor]:
#     X = X_test["X"]
#     A = X_test["A"]
#     Y = X_test["Y"]
#     pi_X = torch.sigmoid(X.mv(theta["propensity_weights"]))
#     mu_X = (
#         X.mv(theta["outcome_weights"])
#         + A * theta["treatment_weight"]
#         + theta["intercept"]
#     )
#     analytic_eif_at_test_pts = (A / pi_X - (1 - A) / (1 - pi_X)) * (Y - mu_X)
#     analytic_correction = analytic_eif_at_test_pts.mean()
#     return analytic_correction, analytic_eif_at_test_pts

### Computing automated doubly robust correction via Monte Carlo

While the doubly robust correction term is known in closed-form for the average treatment effect functional, our `one_step_correction` function in `ChiRho` works for a wide class of other functionals. We focus on the average treatment effect functional here so that we have a ground truth to compare `one_step_correction` against.

In [ ]:
# Helper class to create a trivial guide that returns the maximum likelihood estimate
class MLEGuide(torch.nn.Module):
    def __init__(self, mle_est: ParamDict):
        super().__init__()
        self.names = list(mle_est.keys())
        for name, value in mle_est.items():
            setattr(self, name + "_param", torch.nn.Parameter(value))

    def forward(self, *args, **kwargs):
        for name in self.names:
            value = getattr(self, name + "_param")
            pyro.sample(
                name, pyro.distributions.Delta(value, event_dim=len(value.shape))
            )

# Compute doubly robust ATE estimates using both the automated and closed form expressions
estimators = {"tmle": tmle, "one_step": one_step_corrected_estimator}
estimator_kwargs = {"tmle": {"learning_rate":5e-5,
                "n_grad_steps":500,
                "n_tmle_steps":1,
                "num_nmc_samples":1000,
                "num_grad_samples":N_test}, "one_step": {}}
influences = {"analytic": ATECausalGLM_analytic_influence, "monte_carlo": influence_fn}

estimates = {f"{influence}-{estimator}": torch.zeros(N_datasets) for influence in influences.keys() for estimator in estimators.keys()}
estimates["plug-in-mle"] = torch.zeros(N_datasets)
estimates["plug-in-prior"] = torch.zeros(N_datasets)
estimates["plug-in-truth"] = torch.zeros(N_datasets)

functional = functools.partial(ATEFunctional, num_monte_carlo=10000)

for i in range(N_datasets):
    print("plug-in-prior", i)
    estimates["plug-in-prior"][i] = functional(prior_model)().item()

    print("plug-in-truth", i)
    estimates["plug-in-truth"][i] = functional(true_model)().item()

    D_test = simulated_datasets[i][1]
    theta_hat = fitted_params[i]
    mle_guide = MLEGuide(theta_hat)

    model = PredictiveModel(CausalGLM(p), mle_guide)
    
    print("plug-in-mle", i)
    estimates["plug-in-mle"][i] = functional(model)().detach().item()

    for estimator_str, estimator in estimators.items():
        for influence_str, influence in influences.items():
            # if estimator_str == "tmle" and influence_str == "monte_carlo":
            #     continue

            print(estimator_str, influence_str, i)
            
            estimate = estimator(
                functional, 
                D_test,
                num_samples_outer=max(10000, 100 * p), 
                num_samples_inner=1,
                influence_estimator=influence,
                **estimator_kwargs[estimator_str]
            )(model)()

            estimates[f"{influence_str}-{estimator_str}"][i] = estimate.detach().item()

In [ ]:
results = pd.DataFrame(estimates)

### Results

In [ ]:
# The true treatment effect is 0, so a mean estimate closer to zero is better
results.describe().round(2)

In [ ]:
# Visualize the results
fig, ax = plt.subplots()

for col in results.columns:
    sns.kdeplot(results[col], ax=ax, label=col)

ax.axvline(0, color="black", label="True ATE", linestyle="--")
ax.set_yticks([])
sns.despine()
ax.legend(loc="upper right")
ax.set_xlabel("ATE Estimate")

In [ ]:
plt.scatter(
    results['automated_monte_carlo_correction'],
    results['analytic_correction'],
)
plt.plot(np.linspace(-.2, .5), np.linspace(-.2, .5), color="black", linestyle="dashed")
plt.xlabel("DR-Monte Carlo")
plt.ylabel("DR-Analytic")
sns.despine()

## References

Kennedy, Edward. "Towards optimal doubly robust estimation of heterogeneous causal effects", 2022. https://arxiv.org/abs/2004.14497.